<a href="https://colab.research.google.com/github/zelal-Eizaldeen/DLH_Project_HurtfulWords_Paper/blob/main/Table4_Fairness_metric_recreation_using_preatrained_model_pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Import modules
import sys
sys.path.append('/content/drive/MyDrive/Payel-DLH-related/Colab Notebooks')
import pandas as pd
from pathlib import Path
import os
from transformers import BertForMaskedLM, BertModel, BertTokenizer,AutoModelForMaskedLM, AutoTokenizer
import torch
import pandas as pd

In [ ]:
SCIBERT_DIR = Path('/content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels')

In [ ]:
baseline_model_path = SCIBERT_DIR / 'Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512'

In [ ]:
print(baseline_model_path)

/content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels/Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512


In [ ]:
# DEMOG_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files')
DEMOG_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/DataFiles/cleaned_data/data.csv')
MIMIC_FILE = Path('/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data/phenotyping/train')
MIMIC_FILE_TEST = Path('/content/drive/MyDrive/Payel-DLH-related/mimic3-benchmarks/data/phenotyping/test')

In [ ]:
df_demog = pd.read_csv(DEMOG_FILE)

In [ ]:
print(len(df_demog))

5018


In [ ]:
df_demog.columns

Index(['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input'],
      dtype='object')

In [ ]:
# df_demog=df_demog.reset_index()()
df_demog=df_demog.rename(columns={'row_id':'note_id'})
df_demog['bert_input'].head()

,bert_input
0,"Patient made CMO last night, bradycardic 20-30..."
1,"Patient made CMO last night, bradycardic 20-30..."
2,PHIAGEPHI yr old female admitted to the micu s...
3,PHIAGEPHI yr old female admitted to the micu s...
4,"Comfort care (CMO, Comfort Measures) Assess..."


In [ ]:
df_demog['bert_input'][1]

'Patient made CMO last night, bradycardic 20-30 bpm when I came in,    apneic, appears comfortable. Family at bedside with patient, emotional    support given. Went into asystole, house officer intormed. Pronounced    dead by doctor PHINAMEPHI at  Family offered for autopsy which they    refused to be done to the patient. Post mortem care done, after family    have spent time with patient. Patient s wife not at bedside, children    will inform their mother personally. PHINAMEPHI rites  sacrament of sick    done yesterday. Patient was brought to PHINAMEPHI with name band intact. Comfort care (CMO, Comfort Measures)    Assessment:    Throughout shift pt. became increasingly hypotensive despite being    given blood and albumin. Sats progressively worsened on 100% FIO    Family called and MDs conformed CMO status. Action:    PRN morphine as needed. Morphine gtt ordered but not hung as pt. unresponsive and appears comfortable. Emotional support to family. Response:    Pt. appears comfortabl

In [ ]:
df_mimic = pd.read_csv(MIMIC_FILE/'listfile.csv',index_col=0)
df_mimic = df_mimic.reset_index()

In [ ]:
df_mimic.head(5)

,stay,period_length,Acute and unspecified renal failure,Acute cerebrovascular disease,Acute myocardial infarction,Cardiac dysrhythmias,Chronic kidney disease,Chronic obstructive pulmonary disease and bronchiectasis,Complications of surgical procedures or medical care,Conduction disorders,...,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock,subject_id
0,10004_episode1_timeseries.csv,251.1960,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,10004
1,10004_episode2_timeseries.csv,17.5800,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,10004
2,10014_episode1_timeseries.csv,22.5240,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10014
3,10050_episode1_timeseries.csv,116.6064,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,10050
4,10052_episode1_timeseries.csv,28.6056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10052


In [ ]:

df_mimic['subject_id'] = df_mimic['stay'].str.split('_').str[0]

In [ ]:
df_mimic_test = pd.read_csv(MIMIC_FILE_TEST/'listfile.csv',index_col=0)
df_mimic_test = df_mimic_test.reset_index()

df_mimic_test['subject_id'] = df_mimic_test['stay'].str.split('_').str[0]

In [ ]:
print(len(df_mimic_test))
print(len(df_mimic))

450
2513


In [ ]:
df_demog[['language','language_to_use']].head(100)

,language,language_to_use
0,ENGL,English
1,ENGL,English
2,ENGL,English
3,ENGL,English
4,ENGL,English
...,...,...
95,ENGL,English
96,ENGL,English
97,ENGL,English
98,ENGL,English


In [ ]:
# # df_demog['subject_id_x'] = df_demog['subject_id_x'].astype(int)
# # # df_demog = df_demog[['subject_id_x','gender','ethnicity','language_to_use','insurance']]
# # df_mimic['subject_id'] = df_mimic['subject_id'].astype(int)
# merged_df1 = pd.merge(df_demog, df_mimic, on='subject_id', how='inner')
# merged_df1.head()

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_demog.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input'],
      dtype='object')

In [ ]:
# df_demog=df_demog.rename(columns={'subject_id_x':'subject_id'})
df_demog['subject_id'] = df_demog['subject_id'].astype(int)
# df_demog = df_demog[['subject_id_x','gender','ethnicity','language_to_use','insurance']]
df_mimic['subject_id'] = df_mimic['subject_id'].astype(int)
df_mimic_test['subject_id'] = df_mimic_test['subject_id'].astype(int)
merged_df = pd.merge(df_demog, df_mimic, on='subject_id', how='inner')
merged_df.head()


,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,367228,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 05:18:15,Physician,Intensivist Note,19882.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
merged_df_test = pd.merge(df_demog, df_mimic_test, on='subject_id', how='inner')
merged_df_test.head()

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,460619,50664,178869.0,2170-05-11,2170-05-11 01:11:00,2170-05-11 03:22:23,Nursing,Nursing Progress Note,14657.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,460607,50664,178869.0,2170-05-11,2170-05-11 01:11:00,2170-05-11 01:11:39,Nursing,Nursing Progress Note,14657.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,460616,50664,178869.0,2170-05-11,2170-05-11 01:11:00,2170-05-11 03:08:50,Nursing,Nursing Progress Note,14657.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,475056,98258,129383.0,2127-06-23,2127-06-23 06:35:00,2127-06-23 06:55:35,Nursing,Nursing Progress Note,19611.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,475052,98258,129383.0,2127-06-23,2127-06-23 06:35:00,2127-06-23 06:35:46,Nursing,Nursing Progress Note,19611.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(len(merged_df_test))

52


In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog.csv',index=False)

In [ ]:
merged_df_test.to_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog_test.csv',index=False)

Now that the demographics file (notes) and the phenotypes (labels) have been merges, let's do some more cleaning:

Split each note into sentences, then aggregate short sentences into chunks of at least 20 tokens.

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.data.path.append('/root/nltk_data')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
nltk.data.path.append('/root/nltk_data')

In [ ]:
import os
print(os.listdir('/root/nltk_data/tokenizers'))

['punkt_tab', 'punkt.zip', 'punkt_tab.zip', 'punkt']


In [ ]:
# merged_df = pd.read_csv('/content/drive/MyDrive/Payel-DLH-related/DataFiles/sampled_files/phenotype_and_demog.csv')
# merged_df.head(1)

,note_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,Gastrointestinal hemorrhage,Hypertension with complications and secondary hypertension,Other liver diseases,Other lower respiratory disease,Other upper respiratory disease,Pleurisy; pneumothorax; pulmonary collapse,Pneumonia (except that caused by tuberculosis or sexually transmitted disease),Respiratory failure; insufficiency; arrest (adult),Septicemia (except in labor),Shock
0,367275,21706,108890.0,2116-02-07,2116-02-07 05:18:00,2116-02-07 07:41:39,Physician,Intensivist Note,15420.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
merged_df_copy=merged_df

In [ ]:
merged_df_test_copy=merged_df_test

In [ ]:
print(len(merged_df))

256


In [ ]:
print(len(merged_df_test))

52


In [ ]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Phase 1 tokenization:

Phase 2 tokenization of same train dataset

Phase 2 tokenization of test dataset

In [ ]:
from transformers import BertTokenizer
from tqdm import tqdm
import torch

# Setup
tqdm.pandas()
tokenizer = BertTokenizer.from_pretrained(baseline_model_path)

# Sliding Window Tokenization Function
def sliding_window_tokenize(text, tokenizer, max_len=512, stride=64):
    encoded = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_len,
        stride=stride,
        return_overflowing_tokens=True,
        return_tensors='pt'
    )
    return {
        "input_ids": encoded["input_ids"],              # Shape: [num_chunks, max_len]
        "attention_mask": encoded["attention_mask"]
    }

# Apply per note (from cleaned sentences)
def process_note_with_sliding_window(row, tokenizer, max_len=512, stride=64):
    note = " ".join(row['cleaned_sent'])  # Join all cleaned sentences into one string
    return sliding_window_tokenize(note, tokenizer, max_len=max_len, stride=stride)

# Apply to the DataFrame
merged_df_copy['tokenized_notes'] = merged_df_copy.progress_apply(
    lambda row: process_note_with_sliding_window(row, tokenizer, max_len=512, stride=64),
    axis=1
)


100%|██████████| 256/256 [00:21<00:00, 12.05it/s]


In [ ]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Creating label columns

In [ ]:
label_col = ['Acute and unspecified renal failure',
       'Acute cerebrovascular disease', 'Acute myocardial infarction',
       'Cardiac dysrhythmias', 'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Complications of surgical procedures or medical care',
       'Conduction disorders', 'Congestive heart failure; nonhypertensive',
       'Coronary atherosclerosis and other heart disease',
       'Diabetes mellitus with complications',
       'Diabetes mellitus without complication',
       'Disorders of lipid metabolism', 'Essential hypertension',
       'Fluid and electrolyte disorders', 'Gastrointestinal hemorrhage',
       'Hypertension with complications and secondary hypertension',
       'Other liver diseases', 'Other lower respiratory disease',
       'Other upper respiratory disease',
       'Pleurisy; pneumothorax; pulmonary collapse',
       'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
       'Respiratory failure; insufficiency; arrest (adult)',
       'Septicemia (except in labor)', 'Shock']

In [ ]:
merged_df_copy['labels'] = merged_df_copy[label_col].values.tolist()

In [ ]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

In [ ]:
merged_df_copy['labels'][0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Phase 1 training prep

Phase 2 training prep

In [ ]:
from torch.utils.data import Dataset
import torch

class ClinicalNotesDataset(Dataset):
    def __init__(self, df, max_len=512):
        self.samples = []

        for _, row in df.iterrows():
            # Get tokenized notes and labels
            tokenized = row['tokenized_notes']
            label = torch.tensor(row['labels'], dtype=torch.float)  # Assuming it's a multi-label tensor

            # Extract chunks for each tokenized note
            input_ids_chunks = tokenized['input_ids']  # Shape: [num_chunks, max_len]
            attention_mask_chunks = tokenized['attention_mask']

            # Create a sample for each chunk of the note
            for input_ids, attention_mask in zip(input_ids_chunks, attention_mask_chunks):
                # Repeat the label for every chunk (because label applies to the entire note)
                self.samples.append({
                    'input_ids': input_ids,
                    'attention_mask': attention_mask,
                    'labels': label
                })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return {
            'input_ids': self.samples[idx]['input_ids'],
            'attention_mask': self.samples[idx]['attention_mask'],
            'labels': self.samples[idx]['labels']
        }


Phase 1

Phase 2

In [ ]:
dataset = ClinicalNotesDataset(merged_df_copy)
sample = dataset[0]

print(sample['input_ids'].shape)       # torch.Size([512])
print(sample['attention_mask'].shape)  # torch.Size([512])
print(sample['labels'].shape)          # torch.Size([25])

torch.Size([512])
torch.Size([512])
torch.Size([25])


In [ ]:
train_dataset_phase2 = ClinicalNotesDataset(merged_df_copy, max_len=512)

Phase 1 model training:

Phase 2 model training and evaluating

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score
import numpy as np

# Load the pretrained Phase 1 model
model = BertForSequenceClassification.from_pretrained(baseline_model_path,num_labels=25)

# Define training arguments for evaluation
eval_args = TrainingArguments(
    output_dir="./eval_results",  # You can update this path
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    do_train=False,
    do_eval=True
)

# Custom metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)

    recall = recall_score(labels, preds, average='micro')
    precision = precision_score(labels, preds, average='micro')
    f1 = f1_score(labels, preds, average='micro')
    accuracy = accuracy_score(labels, preds)

    return {
        'recall': recall,
        'precision': precision,
        'f1': f1,
        'accuracy': accuracy,
    }

# Initialize Trainer for evaluation
trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=train_dataset_phase2,
    compute_metrics=compute_metrics
)

# Run evaluation
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# Optional: Get predictions
predictions = trainer.predict(train_dataset_phase2)
y_true = predictions.label_ids
y_pred = (predictions.predictions > 0.5).astype(int)

# Compute specificity
specificities = []
for i in range(y_true.shape[1]):
    cm = confusion_matrix(y_true[:, i], y_pred[:, i])
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificities.append(specificity)
    else:
        specificities.append(np.nan)

average_specificity = np.nanmean(specificities)
print(f"Average Specificity: {average_specificity:.2f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Payel-DLH-related/HurtfulWords/pretrainedModels/Baseline_Clinical_BERT/baseline_clinical_BERT_1_epoch_512 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation Results: {'eval_loss': 0.7049798965454102, 'eval_model_preparation_time': 0.0031, 'eval_recall': 0.10303514376996806, 'eval_precision': 0.3146341463414634, 'eval_f1': 0.1552346570397112, 'eval_accuracy': 0.0, 'eval_runtime': 129.4451, 'eval_samples_per_second': 1.978, 'eval_steps_per_second': 0.124}
Average Specificity: 0.94


In [ ]:
merged_df_copy.columns

Index(['note_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime',
       'storetime', 'category', 'description', 'cgid', 'iserror', 'text',
       'row_id_x', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn',
       'expire_flag', 'fold', 'row_id_y', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'religion',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis',
       'hospital_expire_flag', 'has_chartevents_data', 'dod_merged',
       'ethnicity_to_use', 'age', 'icd9_code', 'language_to_use', 'icustay_id',
       'intime', 'hours_from_icu_admit', 'concat_notes', 'cleaned_sent',
       'bert_input', 'stay', 'period_length',
       'Acute and unspecified renal failure', 'Acute cerebrovascular disease',
       'Acute myocardial infarction', 'Cardiac dysrhythmias',
       'Chronic kidney disease',
       'Chronic obstructive pulmonary disease and bronchiectasis',
       'Com

Table 4 structure formation:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, multilabel_confusion_matrix

# Function to calculate fairness gaps for a binary subgroup vs. the rest (reference group)
def calculate_fairness_vs_others(df, subgroup_col, reference_group, y_true, y_pred):
    # Binary comparison: ref vs all others
    mask_ref = df[subgroup_col] == reference_group
    mask_other = df[subgroup_col] != reference_group

    idx1 = df[mask_ref].index
    idx2 = df[mask_other].index

    if len(idx1) == 0 or len(idx2) == 0:
        print(f"Skipping {subgroup_col} - not enough samples for {reference_group} vs. others")
        return {
            "Fairness Definition": subgroup_col,
            "Comparison (Group 1 vs. Group 2)": f"{reference_group} vs. Other",
            "Recall Gap": np.nan,
            "Parity Gap": np.nan,
            "Specificity Gap": np.nan
        }

    g1_recall, g2_recall = [], []
    g1_parity, g2_parity = [], []
    g1_specificity, g2_specificity = [], []

    # Calculate metrics for each task (column)
    for i in range(y_true.shape[1]):
        r1 = recall_score(y_true[idx1, i], y_pred[idx1, i], zero_division=0)
        r2 = recall_score(y_true[idx2, i], y_pred[idx2, i], zero_division=0)
        g1_recall.append(r1)
        g2_recall.append(r2)

        p1 = y_pred[idx1, i].mean()
        p2 = y_pred[idx2, i].mean()
        g1_parity.append(p1)
        g2_parity.append(p2)

        cm1 = multilabel_confusion_matrix(y_true[idx1, i], y_pred[idx1, i])[0]
        cm2 = multilabel_confusion_matrix(y_true[idx2, i], y_pred[idx2, i])[0]
        tn1, fp1 = cm1[0, 0], cm1[0, 1]
        tn2, fp2 = cm2[0, 0], cm2[0, 1]
        s1 = tn1 / (tn1 + fp1) if (tn1 + fp1) > 0 else 0
        s2 = tn2 / (tn2 + fp2) if (tn2 + fp2) > 0 else 0
        g1_specificity.append(s1)
        g2_specificity.append(s2)

    # Gaps and % favoring reference group
    recall_diffs = np.array(g1_recall) - np.array(g2_recall)
    parity_diffs = np.array(g1_parity) - np.array(g2_parity)
    specificity_diffs = np.array(g1_specificity) - np.array(g2_specificity)

    return {
        "Fairness Definition": subgroup_col,
        "Comparison (Group 1 vs. Group 2)": f"{reference_group} vs. Other",
        "Recall Gap": round(np.mean(np.abs(recall_diffs)), 3),
        "Parity Gap": round(np.mean(np.abs(parity_diffs)), 3),
        "Specificity Gap": round(np.mean(np.abs(specificity_diffs)), 3)
    }

# Main block to calculate fairness gaps for Gender, Language, Ethnicity, and Insurance
results = []

# Gender
results.append(calculate_fairness_vs_others(merged_df_copy, 'gender', 'Male', y_true, y_pred))

# Language
results.append(calculate_fairness_vs_others(merged_df_copy, 'language_to_use', 'English', y_true, y_pred))

# Ethnicity - one row per group
for group in ['White', 'Black', 'Hispanic', 'Asian', 'Other']:
    results.append(calculate_fairness_vs_others(merged_df_copy, 'ethnicity_to_use', group, y_true, y_pred))

# Insurance - one row per group
for group in ['Medicare', 'Private', 'Medicaid']:
    results.append(calculate_fairness_vs_others(merged_df_copy, 'insurance', group, y_true, y_pred))

# Convert to DataFrame for easier table generation
table4_df = pd.DataFrame(results)

# Check the column names to make sure they are correct
print(table4_df.columns)

# Strip any leading/trailing whitespace from column names if necessary
table4_df.columns = table4_df.columns.str.strip()

# Display the DataFrame as a table in Google Colab
table4_df


Skipping gender - not enough samples for Male vs. others
Skipping ethnicity_to_use - not enough samples for White vs. others
Skipping ethnicity_to_use - not enough samples for Black vs. others
Skipping ethnicity_to_use - not enough samples for Hispanic vs. others
Skipping ethnicity_to_use - not enough samples for Asian vs. others
Skipping ethnicity_to_use - not enough samples for Other vs. others
Index(['Fairness Definition', 'Comparison (Group 1 vs. Group 2)', 'Recall Gap',
       'Parity Gap', 'Specificity Gap'],
      dtype='object')


,Fairness Definition,Comparison (Group 1 vs. Group 2),Recall Gap,Parity Gap,Specificity Gap
0,gender,Male vs. Other,NaN,NaN,NaN
1,language_to_use,English vs. Other,0.030,0.033,0.030
2,ethnicity_to_use,White vs. Other,NaN,NaN,NaN
3,ethnicity_to_use,Black vs. Other,NaN,NaN,NaN
4,ethnicity_to_use,Hispanic vs. Other,NaN,NaN,NaN
5,ethnicity_to_use,Asian vs. Other,NaN,NaN,NaN
6,ethnicity_to_use,Other vs. Other,NaN,NaN,NaN
7,insurance,Medicare vs. Other,0.040,0.022,0.040
8,insurance,Private vs. Other,0.050,0.022,0.050
9,insurance,Medicaid vs. Other,0.069,0.058,0.069


In [ ]:
import pandas as pd
import numpy as np

# Assuming y_true and y_pred are NumPy arrays from the predictions
# Let's say you want to inspect the first 5 subjects

indices_to_inspect = [0, 1, 2, 3, 4]  # Example indices of the subjects to check

# Extracting the true and predicted values for those indices
y_true_subset = y_true[indices_to_inspect]
y_pred_subset = y_pred[indices_to_inspect]

# Extract the corresponding subjects from merged_df_test_copy
subjects_subset = merged_df_copy.iloc[indices_to_inspect]

# Combine them into a DataFrame for easier inspection
results_df = subjects_subset.copy()
results_df['y_true'] = [y_true_subset[i] for i in range(len(y_true_subset))]
results_df['y_pred'] = [y_pred_subset[i] for i in range(len(y_pred_subset))]

# Display the results
print(results_df)


   note_id  subject_id   hadm_id   chartdate            charttime  \
0   367275       21706  108890.0  2116-02-07  2116-02-07 05:18:00   
1   367275       21706  108890.0  2116-02-07  2116-02-07 05:18:00   
2   367275       21706  108890.0  2116-02-07  2116-02-07 05:18:00   
3   367228       21706  108890.0  2116-02-07  2116-02-07 05:18:00   
4   367228       21706  108890.0  2116-02-07  2116-02-07 05:18:00   

             storetime   category       description     cgid  iserror  ...  \
0  2116-02-07 07:41:39  Physician  Intensivist Note  15420.0      NaN  ...   
1  2116-02-07 07:41:39  Physician  Intensivist Note  15420.0      NaN  ...   
2  2116-02-07 07:41:39  Physician  Intensivist Note  15420.0      NaN  ...   
3  2116-02-07 05:18:15  Physician  Intensivist Note  19882.0      NaN  ...   
4  2116-02-07 05:18:15  Physician  Intensivist Note  19882.0      NaN  ...   

  Other upper respiratory disease  Pleurisy; pneumothorax; pulmonary collapse  \
0                               0  

In [ ]:
print(merged_df_copy['gender'].value_counts())
print(merged_df_copy['ethnicity_to_use'].value_counts())

gender
M    133
F    123
Name: count, dtype: int64
ethnicity_to_use
WHITE                    185
UNKNOWN/NOT SPECIFIED     39
BLACK                     15
OTHER                      8
HISPANIC/LATINO            7
ASIAN                      2
Name: count, dtype: int64


Extra:










After model 2 finetuning:
 [16/16 12:16, Epoch 1/1]
Epoch	Training Loss	Validation Loss	Recall	Precision	F1	Accuracy
1	No log	0.665566	0.030457	0.083333	0.044610	0.000000
{'eval_loss': 0.6655661463737488, 'eval_recall': 0.030456852791878174, 'eval_precision': 0.08333333333333333, 'eval_f1': 0.04460966542750929, 'eval_accuracy': 0.0, 'eval_runtime': 94.6844, 'eval_samples_per_second': 0.549, 'eval_steps_per_second': 0.042, 'epoch': 1.0}
Test Set Accuracy: 0.0
Test Set F1 Score: 0.04460966542750929
Test Set Recall: 0.030456852791878174
Average Specificity: 0.94
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(




That output isn’t necessarily a code failure, but it does signal a model performance failure — the model barely learned anything useful. Let’s break down what happened and what the messages mean:

🛑 What Went Wrong
🔢 Metrics are very low:
Accuracy: 0.0 (your model didn't get any label combination correct).

Recall: 0.03

Precision: 0.08

F1: 0.04

These are very close to random guessing — so yes, the model’s performance is essentially a failure.

🧠 Likely Causes
Frozen BERT backbone

You're only training the classifier head (you froze all model.bert.parameters()).

If Phase 1 fine-tuning didn't give you good domain-specific representations, freezing BERT in Phase 2 might leave the classifier powerless to adapt to new patterns.

✅ Fix: Try unfreezing some or all of the BERT layers for Phase 2 and fine-tune the whole model.

Too little training (1 epoch)

One epoch might not be enough, especially with a small learning head and a frozen encoder.

✅ Fix: Try more epochs (e.g., 3–5) and monitor validation F1 after each.

Bad threshold

Threshold 0.5 might be too strict — possibly resulting in very few positive predictions.

✅ Fix: Try threshold tuning (e.g., test thresholds from 0.1 to 0.9 and pick the best based on validation F1).

Label imbalance

If some labels are rare, metrics (especially F1/recall) can be skewed badly.

✅ Fix: Consider:

Oversampling underrepresented labels.

Using a weighted loss like BCEWithLogitsLoss(pos_weight=...).

⚠️ About the Warning:
plaintext
Copy
Edit
UserWarning: A single label was found in 'y_true' and 'y_pred'.
This means that for some labels, either the model predicted only 0s (no positives), or your test labels had only 0s. As a result, the confusion matrix had only one class — which confuses confusion_matrix().

✅ Fix: When computing confusion matrices in multi-label setups, pass all possible labels explicitly:

python
Copy
Edit
confusion_matrix(y_true[:, i], y_pred[:, i], labels=[0, 1])
✅ Summary
This isn’t a bug, but it's a sign the Phase 2 training isn’t effective yet. To improve:

Try unfreezing the BERT layers.

Train longer.

Tune the threshold.

Handle label imbalance better.

Make sure your datasets are clean and correctly labeled.

Want help modifying the code to unfreeze BERT or run threshold tuning?